# Load data

In [1]:
import pandas as pd

/tmp/ipykernel_21676/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
from pathlib import Path

def load_data(path: str) -> pd.DataFrame:
    data = pd.read_csv(Path(path))
    return data

In [3]:
data = load_data("../data/data.csv")

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


# Preprocessing

In [5]:
from warnings import filterwarnings
filterwarnings("ignore")

In [6]:
from sentence_transformers import SentenceTransformer

# intfloat/e5-small-v2 or intfloat/e5-base-v2 or intfloat/e5-large-v2
model = SentenceTransformer("intfloat/e5-small-v2")

In [7]:
embeddings = model.encode(data['text'], normalize_embeddings=True, show_progress_bar=True)

Batches:   0%|          | 0/238 [00:00<?, ?it/s]

In [ ]:
nb_comp = len(embeddings[0])
nb_data = len(embeddings)
print(f"embedding size : {nb_comp}")

In [ ]:
for num_comp in range(nb_comp):
    data['x_' + str(num_comp)] = [embeddings[k][num_comp] for k in range(nb_data)]

In [ ]:
data.head(10)

In [ ]:
data_keywords = data['keyword']

In [ ]:
data_keywords.size

In [ ]:
data_keywords.value_counts()

In [ ]:
columns = ['keyword']
data_dummy = pd.get_dummies(data, columns=columns, drop_first=True)
data_dummy.replace({False: 0, True: 1}, inplace=True)

In [ ]:
data_dummy

In [ ]:
data_dummy.info()

In [ ]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="my_app")
data_dummy_wNan = data_dummy.dropna()

def get_coordinates(location):
    try:
        location = geolocator.geocode(location)
        return location.latitude, location.longitude
    except:
        return None

data_tmp = data_dummy_wNan.head(10)
for location in data_tmp['location']:
    print(location, get_coordinates(location))
# data_tmp['coordinates'] = data1['location'].apply(get_coordinates)


In [ ]:
data_dummy_wNan.apply(lambda x: get_coordinates(x['location']), axis=1)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

data_dummy_wNan = data_dummy.dropna()
corr = data_dummy_wNan.corr()
sns.heatmap(corr, annot=False, cmap='coolwarm', linewidths=0.5)

